# meiosis

> Fill in a module description here

In [ ]:
#| default_exp meiosis

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import jax
import jax.numpy as jnp
from chewc.sp import SimParam
from chewc.popgen import quick_haplo
from chewc.population import Population
from chewc.trait import add_trait_a, TraitCollection
from chewc.phenotype import set_pheno

# --- Imports for Testing ---
from fastcore.test import test_eq, test_close, test_fail
import jax
import jax.numpy as jnp
import numpy as np

# --- Functions from other modules needed for testing ---
from chewc.population import Population
from chewc.sp import SimParam

from jax import lax, vmap
from functools import partial
import matplotlib.pyplot as plt

In [ ]:
#| export



@partial(jax.jit, static_argnames=("max_crossovers",))
def _sample_chiasmata(key: jax.random.PRNGKey, 
                      map_length: float, 
                      v: float, 
                      max_crossovers: int = 10) -> jnp.ndarray:
    """
    Generates crossover positions along a single chromosome using a Gamma process
    to model chiasma interference.

    This function is a core component of the meiosis simulation and is designed
    to be highly efficient on accelerators through JIT compilation.

    --- JAX Implementation Notes ---
    This function demonstrates several key JAX patterns for handling complex,
    stateful, and seemingly dynamic processes in a purely functional way.

    1.  **`lax.scan` for Loops**: A standard Python `for` loop cannot be JIT-compiled
        if its length is not a compile-time constant. `lax.scan` is used here
        to perform a sequential operation (generating one chiasma after another)
        in a way that JAX can compile into an efficient, recurrent kernel. It
        carries state (`key` and `last_pos`) between loop iterations functionally.

    2.  **Static Arguments**: The `max_crossovers` parameter determines the output
        shape of `lax.scan`. Since all array shapes must be static (known at
        compile-time), this argument MUST be marked as static using the
        `@partial(jax.jit, static_argnames=...)` decorator. This tells JAX to
        re-compile the function if this value changes, which is the desired
        behavior for simulation parameters.

    3.  **Fixed-Size Output and Masking**: JAX functions must return arrays of a
        fixed shape. To handle a variable number of crossovers, this function
        first generates a fixed-size array of potential positions (of length
        `max_crossovers`). It then uses a boolean mask and `jnp.where` to
        filter out invalid positions (those outside the chromosome bounds),
        replacing them with `jnp.nan`. This "generate-and-mask" approach is a
        standard and highly efficient JAX pattern.

    Args:
        key: A JAX random key.
        map_length: The genetic length of the chromosome (e.g., in Morgans).
        v: The interference parameter from the Gamma distribution (nu).
        max_crossovers: The maximum number of crossovers to simulate. This
                        must be a static integer.

    Returns:
        A fixed-size JAX array of shape (`max_crossovers`,) containing the
        positions of valid crossovers. Invalid or non-existent crossover
        slots are filled with `jnp.nan`.
    """
    shape = v
    scale = 1.0 / (2.0 * v)

    def scan_body(carry, _):
        key, last_pos = carry
        key, subkey = jax.random.split(key)
        distance = jax.random.gamma(subkey, shape) * scale
        new_pos = last_pos + distance
        return (key, new_pos), new_pos

    key, initial_key = jax.random.split(key)
    initial_start_pos = jax.random.uniform(initial_key, minval=-10.0, maxval=0.0)
    init_carry = (key, initial_start_pos)
    
    _, crossover_positions = lax.scan(scan_body, init_carry, None, length=max_crossovers)
    
    # --- CORRECTED LOGIC ---
    # A crossover is only valid if it's within the bounds (0, map_length).
    valid_mask = (crossover_positions > 0) & (crossover_positions < map_length)
    valid_crossovers = jnp.where(valid_mask, 
                                 crossover_positions, 
                                 jnp.nan)
    
    return valid_crossovers


In [ ]:
#| export
@partial(jax.jit, static_argnames=("max_crossovers",))
def _create_gamete(key: jax.random.PRNGKey, 
                   parental_haplotypes: jnp.ndarray,
                   gen_map: jnp.ndarray,
                   v_interference: float,
                   max_crossovers: int = 20) -> jnp.ndarray:
    """
    Creates a single recombinant gamete from a parent's two haplotypes for one
    chromosome.

    --- JAX Implementation Notes ---

    This function is a masterclass in vectorized computation and is the core
    of the recombination logic. It avoids all Python loops to remain highly
    performant under JIT compilation.

    1.  **Static Argument Propagation**: This function calls `_sample_chiasmata`,
        which requires `max_crossovers` to be a static argument. Therefore,
        this function must also treat `max_crossovers` as static, ensuring
        a valid call chain for the JIT compiler.

    2.  **Vectorized Recombination via `searchsorted`**: The core of this
        function is a clever, vectorized method to determine which parental
        haplotype to use for each locus:
        a. It first samples a fixed-size array of potential crossover locations.
        b. `jnp.searchsorted` is used to count how many crossovers have occurred
           before each locus on the chromosome. This creates an array of "segment
           indices" (e.g., 0 for the first segment, 1 for the second, etc.).
        c. Modulo-2 arithmetic (`% 2`) is applied to these segment indices to
           create a final haplotype choice mask (0 or 1) that alternates at
           each crossover point.

    3.  **`jnp.where` for Final Selection**: The final gamete is constructed
        using `jnp.where`, which efficiently selects alleles from one of the two
        parental haplotypes based on the choice mask generated in the previous
        step.

    Args:
        key: A JAX random key.
        parental_haplotypes: A (2, nLoci) array of the two parental haplotypes.
        gen_map: A (nLoci,) array of locus positions.
        v_interference: The interference parameter for chiasma sampling.
        max_crossovers: A static integer for the maximum number of crossovers.

    Returns:
        A (nLoci,) array representing the new, recombinant gamete.
    """
    key, chiasma_key, hap_key = jax.random.split(key, 3)

    map_length = gen_map[-1]
    
    crossover_positions = _sample_chiasmata(chiasma_key, map_length, 
                                            v_interference, max_crossovers)

    crossover_indices = jnp.searchsorted(gen_map, crossover_positions)

    start_hap = jax.random.choice(hap_key, jnp.array([0, 1], dtype=jnp.uint8))

    n_loci = gen_map.shape[0]
    
    # --- FIX ---
    # Use side='right' to ensure the crossover happens at the correct locus index.
    locus_segments = jnp.searchsorted(crossover_indices, jnp.arange(n_loci), side='right')
    
    haplotype_choice = (start_hap + locus_segments) % 2
    
    new_gamete = jnp.where(haplotype_choice == 0, 
                           parental_haplotypes[0], 
                           parental_haplotypes[1])
    
    return new_gamete


In [ ]:
#| export
# The signature and decorator are changed to be more specific and JIT-friendly.
@partial(jax.jit, static_argnames=("n_chr",))
def meiosis_for_one_cross(key: jax.random.PRNGKey,
                          mother_geno: jnp.ndarray,
                          father_geno: jnp.ndarray,
                          n_chr: int,
                          gen_map: jnp.ndarray,
                          v_interference: float
                         ) -> jnp.ndarray:
    """
    Creates a single diploid progeny's genotype from two parents' genotypes
    by simulating meiosis for all chromosomes in parallel.

    This function is a high-performance kernel designed to be compiled by JAX.

    --- JAX Implementation Notes ---
    This function showcases the composition of JAX's core transformations for
    maximum performance on parallel architectures (like GPUs).

    1.  **`vmap` for Parallelism**: The core logic of creating a gamete for a
        single chromosome is defined in `_create_gamete`. `vmap` is used to
        automatically "vectorize" this function, applying it across all
        chromosomes of a parent simultaneously. The `in_axes` argument
        is critical for this:
        - `in_axes=(0, 0, 0, None)` tells `vmap` to map over the first axis
          (the chromosome axis) of the `keys`, `parent_geno`, and `gen_map`
          arrays, while broadcasting the single `v_interference` value to all
          parallel executions. This avoids unnecessary memory duplication.

    2.  **`jit` for Fused Compilation**: The entire function is JIT-compiled.
        JAX is able to "fuse" the `vmap` operations and the final `jnp.stack`
        into a single, highly-optimized kernel. This minimizes overhead from
        launching separate computations and maximizes hardware utilization.

    3.  **Static Arguments**: The number of chromosomes, `n_chr`, is used to
        determine the number of random keys to split via
        `jax.random.split(key, n_chr)`. Because the shape of a JAX array must
        be known at compile time, `n_chr` cannot be a dynamic (traced) value.
        It is therefore marked as a static argument, meaning JAX will
        re-compile this function if `n_chr` changes.

    Args:
        key: A JAX random key.
        mother_geno: The mother's genotype. Shape: `(nChr, ploidy, nLoci)`.
        father_geno: The father's genotype. Shape: `(nChr, ploidy, nLoci)`.
        n_chr: The number of chromosomes. **Must be a static integer.**
        gen_map: The genetic map defining locus positions for each chromosome.
        v_interference: The interference parameter for the Gamma process.

    Returns:
        The progeny's complete diploid genotype. Shape: `(nChr, ploidy, nLoci)`.
    """

    key_mother, key_father = jax.random.split(key)

    # Define a vectorized version of the single-chromosome gamete creator.
    vmapped_gamete_creator = vmap(
        _create_gamete,
        in_axes=(0, 0, 0, None)  # Map over keys, geno, gen_map; broadcast v
    )

    # Create all of the mother's gamete-haplotypes in parallel
    mother_gametes = vmapped_gamete_creator(
        jax.random.split(key_mother, n_chr),
        mother_geno,
        gen_map,
        v_interference
    )

    # Create all of the father's gamete-haplotypes in parallel
    father_gametes = vmapped_gamete_creator(
        jax.random.split(key_father, n_chr),
        father_geno,
        gen_map,
        v_interference
    )

    # Stack the two resulting gametes to form the new diploid genotype
    progeny_geno = jnp.stack([mother_gametes, father_gametes], axis=1)

    return progeny_geno

In [ ]:
#| export

@partial(jax.jit, static_argnames=("max_crossovers",))
def _create_gamete(key: jax.random.PRNGKey, 
                   parental_geno_haplotypes: jnp.ndarray,
                   parental_ibd_haplotypes: jnp.ndarray,  # NEW: IBD haplotypes
                   gen_map: jnp.ndarray,
                   v_interference: float,
                   max_crossovers: int = 20) -> tuple[jnp.ndarray, jnp.ndarray]:  # Return both geno and IBD
    """
    Creates a single recombinant gamete from a parent's two haplotypes for one
    chromosome, tracking both genotypes and IBD.

    Args:
        key: A JAX random key.
        parental_geno_haplotypes: A (2, nLoci) array of the two parental genotype haplotypes.
        parental_ibd_haplotypes: A (2, nLoci) array of the two parental IBD haplotypes.
        gen_map: A (nLoci,) array of locus positions.
        v_interference: The interference parameter for chiasma sampling.
        max_crossovers: A static integer for the maximum number of crossovers.

    Returns:
        A tuple of (new_geno_gamete, new_ibd_gamete), both (nLoci,) arrays.
    """
    key, chiasma_key, hap_key = jax.random.split(key, 3)

    map_length = gen_map[-1]
    
    crossover_positions = _sample_chiasmata(chiasma_key, map_length, 
                                            v_interference, max_crossovers)

    crossover_indices = jnp.searchsorted(gen_map, crossover_positions)

    start_hap = jax.random.choice(hap_key, jnp.array([0, 1], dtype=jnp.uint8))

    n_loci = gen_map.shape[0]
    
    # Use side='right' to ensure the crossover happens at the correct locus index.
    locus_segments = jnp.searchsorted(crossover_indices, jnp.arange(n_loci), side='right')
    
    haplotype_choice = (start_hap + locus_segments) % 2
    
    # Apply THE SAME choice mask to both genotypes and IBD
    new_geno_gamete = jnp.where(haplotype_choice == 0, 
                                parental_geno_haplotypes[0], 
                                parental_geno_haplotypes[1])
    
    new_ibd_gamete = jnp.where(haplotype_choice == 0,
                               parental_ibd_haplotypes[0],
                               parental_ibd_haplotypes[1])
    
    return new_geno_gamete, new_ibd_gamete


@partial(jax.jit, static_argnames=("n_chr",))
def meiosis_for_one_cross(key: jax.random.PRNGKey,
                          mother_geno: jnp.ndarray,
                          father_geno: jnp.ndarray,
                          mother_ibd: jnp.ndarray,   # NEW: Mother IBD
                          father_ibd: jnp.ndarray,   # NEW: Father IBD
                          n_chr: int,
                          gen_map: jnp.ndarray,
                          v_interference: float
                         ) -> tuple[jnp.ndarray, jnp.ndarray]:  # Return both geno and IBD
    """
    Creates a single diploid progeny's genotype and IBD from two parents
    by simulating meiosis for all chromosomes in parallel.

    Args:
        key: A JAX random key.
        mother_geno: The mother's genotype. Shape: `(nChr, ploidy, nLoci)`.
        father_geno: The father's genotype. Shape: `(nChr, ploidy, nLoci)`.
        mother_ibd: The mother's IBD array. Shape: `(nChr, ploidy, nLoci)`.
        father_ibd: The father's IBD array. Shape: `(nChr, ploidy, nLoci)`.
        n_chr: The number of chromosomes. **Must be a static integer.**
        gen_map: The genetic map defining locus positions for each chromosome.
        v_interference: The interference parameter for the Gamma process.

    Returns:
        A tuple of (progeny_geno, progeny_ibd), both with shape `(nChr, ploidy, nLoci)`.
    """

    key_mother, key_father = jax.random.split(key)

    # Define a vectorized version of the single-chromosome gamete creator.
    vmapped_gamete_creator = vmap(
        _create_gamete,
        in_axes=(0, 0, 0, 0, None)  # Map over keys, geno, ibd, gen_map; broadcast v
    )

    # Create all of the mother's gametes in parallel
    mother_geno_gametes, mother_ibd_gametes = vmapped_gamete_creator(
        jax.random.split(key_mother, n_chr),
        mother_geno,
        mother_ibd,
        gen_map,
        v_interference
    )

    # Create all of the father's gametes in parallel
    father_geno_gametes, father_ibd_gametes = vmapped_gamete_creator(
        jax.random.split(key_father, n_chr),
        father_geno,
        father_ibd,
        gen_map,
        v_interference
    )

    # Stack the two resulting gametes to form the new diploid genotype and IBD
    progeny_geno = jnp.stack([mother_geno_gametes, father_geno_gametes], axis=1)
    progeny_ibd = jnp.stack([mother_ibd_gametes, father_ibd_gametes], axis=1)

    return progeny_geno, progeny_ibd

In [ ]:
#| test


# --- Test Setup Helper ---

def _setup_meiosis_test():
    """Provides a consistent setup for meiosis tests."""
    key = jax.random.PRNGKey(123)
    n_ind = 2
    n_chr = 5
    n_loci_per_chr = 1000  # Use a larger number of loci to ensure crossovers happen
    ploidy = 2

    # Create a base population and SimParam
    founder_pop, gen_map = quick_haplo(key, n_ind, n_chr, n_loci_per_chr, ploidy=ploidy)
    sp = SimParam.from_founder_pop(founder_pop, gen_map, v_interference=2.6)

    # Create distinct parents
    # Mother is all zeros, Father is all ones
    mother_geno = jnp.zeros_like(founder_pop.geno[0])
    father_geno = jnp.ones_like(founder_pop.geno[1])
    
    return {
        "key": key,
        "mother_geno": mother_geno,
        "father_geno": father_geno,
        "sp": sp,
        "gen_map": gen_map
    }

In [ ]:
#| test

def test_sample_chiasmata_shape_and_bounds():
    "Tests the output shape and that positions are within the chromosome."
    # ARRANGE
    setup = _setup_meiosis_test()
    key = setup["key"]
    sp = setup["sp"]
    max_crossovers = 20
    map_length = setup["gen_map"][0, -1] # Length of one chromosome

    # ACT
    crossovers = _sample_chiasmata(key, map_length, sp.recomb_params[0], max_crossovers=max_crossovers)

    # ASSERT
    test_eq(crossovers.shape, (max_crossovers,))
    
    # Check that all non-NaN values are within the valid range
    valid_crossovers = crossovers[~jnp.isnan(crossovers)]
    assert jnp.all(valid_crossovers > 0)
    assert jnp.all(valid_crossovers < map_length)

def test_sample_chiasmata_interference():
    "Tests the statistical properties of the interference parameter 'v'."
    # ARRANGE
    key = jax.random.PRNGKey(42)
    n_reps = 1000
    map_length = 1.0  # 1 Morgan
    
    keys = jax.random.split(key, n_reps)

    # --- With interference (v > 1) ---
    v_interference = 5.0 
    sample_fn_interference = jax.vmap(_sample_chiasmata, in_axes=(0, None, None))
    all_crossovers_interference = sample_fn_interference(keys, map_length, v_interference)
    n_crossovers_interference = jnp.sum(~jnp.isnan(all_crossovers_interference), axis=1)
    
    # --- Without interference (v = 1, approximates a Poisson process) ---
    v_no_interference = 1.0
    sample_fn_no_interference = jax.vmap(_sample_chiasmata, in_axes=(0, None, None))
    all_crossovers_no_interference = sample_fn_no_interference(keys, map_length, v_no_interference)
    n_crossovers_no_interference = jnp.sum(~jnp.isnan(all_crossovers_no_interference), axis=1)

    # ASSERT
    # Haldane's mapping function (no interference) expects lambda = 2 * map_length crossovers
    # Our model is slightly different, but the mean should be close to map_length.
    # The key is that interference should reduce the variance and mean number of crossovers.
    mean_xo_interference = jnp.mean(n_crossovers_interference)
    mean_xo_no_interference = jnp.mean(n_crossovers_no_interference)
    
    assert mean_xo_interference < mean_xo_no_interference
    test_close(mean_xo_no_interference, map_length, eps=0.1)

In [ ]:
#| test

def test_create_gamete_recombination():
    "Tests that a gamete is a valid recombinant of parental haplotypes."
    # ARRANGE
    setup = _setup_meiosis_test()
    key = setup["key"]
    sp = setup["sp"]
    
    # Parental haplotypes are distinct: [all 0s, all 1s]
    parental_haps = jnp.stack([
        jnp.zeros(sp.n_loci_per_chr[0], dtype=jnp.uint8),
        jnp.ones(sp.n_loci_per_chr[0], dtype=jnp.uint8)
    ])
    
    # ACT (with recombination)
    gamete = _create_gamete(key, parental_haps, setup["gen_map"][0], sp.recomb_params[0])
    
    # ASSERT (with recombination)
    test_eq(gamete.shape, (sp.n_loci_per_chr[0],))
    # A successful recombination should have both 0s and 1s
    assert jnp.any(gamete == 0)
    assert jnp.any(gamete == 1)
    
    # ACT (without recombination)
    # Create a genetic map with zero length to prevent crossovers
    zero_map = jnp.zeros_like(setup["gen_map"][0])
    gamete_no_recomb = _create_gamete(key, parental_haps, zero_map, sp.recomb_params[0])
    
    # ASSERT (without recombination)
    # The gamete should be identical to one of the parental haplotypes
    is_hap0 = jnp.all(gamete_no_recomb == parental_haps[0])
    is_hap1 = jnp.all(gamete_no_recomb == parental_haps[1])
    assert is_hap0 or is_hap1


def test_meiosis_for_one_cross_output():
    "Tests the final output shape and allele origins for a full cross."
    # ARRANGE
    setup = _setup_meiosis_test()
    key = setup["key"]
    sp = setup["sp"]
    
    # ACT
    progeny_geno = meiosis_for_one_cross(
        key,
        setup["mother_geno"],
        setup["father_geno"],
        sp.n_chr,
        setup["gen_map"],
        sp.recomb_params[0]
    )
    
    # ASSERT
    # Check shape: (nChr, ploidy, nLoci)
    expected_shape = (sp.n_chr, sp.ploidy, sp.n_loci_per_chr[0])
    test_eq(progeny_geno.shape, expected_shape)
    
    # Check allele origin. One haplotype in the progeny should be all 0s (from mother)
    # and the other should be all 1s (from father). Recombination happens within
    # a parent before segregation, so this should hold true.
    maternal_haplotype = progeny_geno[:, 0, :]
    paternal_haplotype = progeny_geno[:, 1, :]
    
    assert jnp.all(maternal_haplotype == 0)
    assert jnp.all(paternal_haplotype == 1)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()